## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [148]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import matplotlib.pyplot as plt

# Sampling frequency said
fs = 125


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def RunPulseRateAlgorithm(data_fl, ref_fl):
    '''Takes the data and the reference one and compares the values to measure the error and the confidence.
    Both are returned.'''
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    bpm = sp.io.loadmat(ref_fl)['BPM0']
    
    # Compute pulse rate estimates and estimation confidence.
    ppg = bandpass_filter(ppg)
    accx = bandpass_filter(accx)
    accy = bandpass_filter(accy)
    accz = bandpass_filter(accz)
    
    # Take the frequency of the accelerometer by combining the three signals
    acc = (accx**2 + accy**2 + accz**2)**(1/2)
    
    window_shift = 2*fs
    window_length = 8*fs
    
    bpms = list()
    confidence_list = list()
    errors_list = list()
    # We start with a previous of 100 which is a common value
    previous = 100
    for i in range(0, len(ppg)-window_length, window_shift):
        part_ppg = ppg[i:i+window_length]
        part_acc = acc[i:i+window_length]
    
        freq, freqs, fft = heart_rate_calc(ppg=part_ppg, acc=part_acc, fs=fs, prev = previous)
        previous = freq
        confidence_list.append(confidence_calc(freqs, fft, freq))
        bpms.append(freq)
    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    confidence = np.array(confidence_list)
    errors = np.array(np.abs(np.diag(np.subtract(bpms, bpm))))
    return errors, confidence

In [155]:
import scipy.signal

def bandpass_filter(signal, band_pass = (70/60, 200/60), fs = fs):
    """ Performs a bandpass filter on the signal. """
    
    b,a = scipy.signal.butter(3, band_pass, fs=fs, btype= 'bandpass')
    
    # Perform forward and backward digital butterworth filter
    return scipy.signal.filtfilt(b,a,signal)

def fourier_transform(signal, fs):
    '''Calculates the fourier transform for the signal (frequencies + fft)'''
    freqs = np.fft.rfftfreq(len(signal), 1/fs)
    fft = np.abs(np.fft.rfft(signal))
    return freqs, fft

def heart_rate_calc(ppg, acc, fs, prev):
    
    """Takes the ppg, acc, fs and the previous frequency and calculates the heart rate depending on the
    Fourier Transform and the previous frequency. It returns the frequency measured and the fft and frequencies of FFT."""
    
    # Take the peaks of the fft
    freq_ppg, fft_ppg = fourier_transform(ppg, fs)
    
    # Delete the fft at frequencies out of the range 70bpm to 200bpm
    fft_ppg[freq_ppg <= 70/60.0] = 0.0
    fft_ppg[freq_ppg >= 200/60.0] = 0.0
    
    # Calculate the peaks
    peaks_ppg = sp.signal.find_peaks(fft_ppg)[0]
    freq_peaks_ppg = freq_ppg[peaks_ppg]
    
    # Same with accelerometer signal
    freq_acc, fft_acc = fourier_transform(acc, fs)
    
    # Delete the fft at frequencies out of the range 70bpm to 200bpm
    fft_acc[freq_acc <= 70/60.0] = 0.0
    fft_acc[freq_acc >= 200/60.0] = 0.0
    
    # Calculate the peaks
    peaks_acc = sp.signal.find_peaks(fft_acc)[0]
    freq_peaks_acc = freq_acc[peaks_acc]
    
    # The limit of frequency from the previous point is 11 bpm
    limit = 11/60
    delta = 2/60 #considered in some scientific papers
    
    #Create a dictionary with frequencies and each magnitude and sort aftwerwards for ppg and accelerometer
    ppg_dict = dict(zip(freq_ppg, fft_ppg))
    sort_ppg_dict = dict(sorted(ppg_dict.items(), key=lambda item: item[1], reverse=True))
    
    acc_dict = dict(zip(freq_ppg, fft_ppg))
    sort_acc_dict = dict(sorted(acc_dict.items(), key=lambda item: item[1], reverse=True))
    
    # Take the frequency without bpm
    prev = prev/60
    
    # Calculates the frequency depending on the limit set between ppg and accelerometer
    #if np.abs(list(sort_ppg_dict.keys())[0] - list(sort_acc_dict.keys())[0]) > limit:
    if np.abs(list(sort_ppg_dict.keys())[0] - prev) > limit:
        dif = list(sort_ppg_dict.keys())[0] - prev
        if dif>0:
            sel_freq = prev+delta
        else:
            sel_freq = prev-delta
    else:
        sel_freq = list(sort_ppg_dict.keys())[0]
    
    # If they are similar both frequencies, it takes the six biggest and measures the frequency
    #else:
        
        #keys_sort_list = list(sort_ppg_dict.keys())
        #print([i*60 for i in [np.abs(prev-keys_sort_list[i]) for i in range(1,8)]])
        #argument = np.argmin(np.array([np.abs(prev-keys_sort_list[i]) for i in range(1,6)]))
        #sel_freq = list(sort_ppg_dict.keys())[argument+1]

    sel_freq = sel_freq*60
    
    return sel_freq, freq_ppg, fft_ppg

def confidence_calc(freqs, fft_mag, bpm_f):
    '''Function that calculates the cofidence interval taking those ones which are 15 bpm '''
    freqs = freqs*fs
    window_f = 15/60
    fundamental_freq_window = (freqs > bpm_f - window_f) & (freqs < bpm_f + window_f)
    return np.sum(fft_mag[fundamental_freq_window])/ np.sum(fft_mag)

In [156]:
# Load Data and calculate the first one
data_fls, ref_fls = LoadTroikaDataset()
data_fl, ref_fl = data_fls[0], ref_fls[0]
bpm = sp.io.loadmat(ref_fl)['BPM0']
err, conf = RunPulseRateAlgorithm(data_fls[0], ref_fls[0])

In [157]:
Evaluate()

12.70478220689467

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

**Code Description:** the algorithm takes the frequency and measures the previous one, and looks if it diffears too much from the previous. This is done with a moving windows of length 8s and 2s shift. If the difference from the previous one is too high, it applies a change of a delta 2bpm which is considered by some scientists. Finally it measures the error and the confidence of the prediction.

**Data Description:** data used consists on a set with ppg signal and accelerometers at the three axis. It also contains the reference bpm to calculate the error of our algorithm. The ground truth for the reference bpm is ECG. The dataset contains information of 12 patients from 18 to 25 years old doing intensive physical exercise from wrist-type photoplethysmographic signals. The sampling rate of the signal is 125 Hz.

The short-commings of the dataset are the low amount of data which can produce to overfit the algorithm and determine an error which not corresponds to reality.

**Algorithm Description:** first a bandpass filter is used of 70 to 200 bpm (which is more realistic and offers better results, with less error). Afterwards a moving windows takes 8s of signal and measures the most important frequencies that, if it's different to the previous one by 11 bpm takes a delta value (of 2 bpm) depending if the difference is positive or negative, if not, it takes that one because is considered MAE is not increasing. The evaluation is done with the mean absolute error which had to be lower than 15 bpm.

If the frequency of the ppg is similar to the accelerometer's one, this will be confused and the error will increase. Other caveats can be the presence of noise in the PPG which is not visible in the accelerometer, so it could not be seen and will produce algorithm fail.

Confidence tells how much energy in the frequency spectrum is concentrated near the pulse rate estimated. So higher value tells the estimate is better.

**Algorithm Performance:** the mean absolute error is measured and compared to a reference list measured with an ECG. The MAE takes the absolute value of the difference between the reference value and the estimated one, and finally makes the mean for all the values. It's a way to estimate how is performing our algorithm.

In this case, the reference values (ground truth) were calculated with 8s windows of an ECG. By steps of 2s, values were measured and then compared obtaining a value of 12.7 bpm which is lower than 15 bpm (our goal).

-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 